Источник: https://github.com/PacktPublishing/Hands-On-Image-Processing-with-Python/tree/master

In [37]:
# !pip install -U scikit-image

In [38]:
import os
from PIL import Image, ImageFilter
import numpy as np
from numpy import asarray
import shutil
from scipy.ndimage import gaussian_filter
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
from numpy import linalg as LA
from skimage import exposure
import warnings
warnings.filterwarnings("ignore")

In [39]:
# constants

# 1
P = 256

# 4
K = 5

In [40]:
# functions
def remove(path):
    if os.path.exists(path):  
        if os.path.isfile(path) or os.path.islink(path):
            os.unlink(path)
        else:
            shutil.rmtree(path)

def contrast_stretching(img: np.array) -> np.array:
    return ((img - img.min()) / (img.max() - img.min()) * (P-1)).astype(np.uint8)

In [79]:
for i in range(1, 8):
    remove(f"files/{i}")
for i in range(1, 8):
    os.mkdir(f"files/{i}")


img = Image.open("files/image.jpg")

### 1. Загрузка и визуализация изображения
#### a) Числовые характеристики изображения

In [42]:
print(f"Размерность изображения: {img.size[0]}x{img.size[1]}")
print(f"Число световых каналов: {len(img.getbands())}")
print(f"Яркостное разрешение: {len(img.getbands()) * 8} bits")


Размерность изображения: 1600x1060
Число световых каналов: 3
Яркостное разрешение: 24 bits


#### б) Если изображение содержит много цветовых каналов, то привести его к одноканальному (в градациях серого) в диапазоне [0; P], P = 255 (яркостное разрешение – 1 байт) 

In [43]:
gray = img.convert('L')
gray.save('files/greyscale.jpg')

img_array = asarray(gray)

### 2. Градиентные операторы

#### a) Рассчитать градиентные изображения в вертикальном и горизонтальном направлениях, а также изображения модулей и углов градиентов, используя оператор Робертса
Источник: https://en.wikipedia.org/wiki/prewitt_cross

Источник: https://habr.com/ru/articles/114452/

In [64]:
roberts_cross_v = np.array( [[1, 0 ], 
                             [0,-1 ]] ) 
  
roberts_cross_h = np.array( [[ 0, 1 ], 
                             [ -1, 0 ]] )

g_vertical = cv2.filter2D(img_array, -1, roberts_cross_v)
exp_g_vertical = cv2.equalizeHist(g_vertical)
Image.fromarray(exp_g_vertical).convert('L').save('files/1/roberts_vertical.jpg')

g_horizontal = cv2.filter2D(img_array, -1, roberts_cross_h) 
exp_g_horizontal = cv2.equalizeHist(g_horizontal)
Image.fromarray(exp_g_horizontal).convert('L').save('files/1/roberts_horizontal.jpg')

g_module = np.sqrt( np.square(g_horizontal) + np.square(g_vertical)).astype(np.uint8)
exp_g_module = cv2.equalizeHist(g_module)
Image.fromarray(exp_g_module).convert('L').save('files/1/roberts_module.jpg')

g_angle = (np.arctan(g_vertical/g_horizontal) - 0.75 * np.pi).astype(np.uint8)
exp_g_angle = cv2.equalizeHist(g_angle)
Image.fromarray(exp_g_angle).convert('L').save('files/1/roberts_angle.jpg')

In [65]:
prewitt_cross_v = np.array( [[1, 0, -1 ], 
                             [1, 0, -1],
                             [1, 0, -1]]) 
  
prewitt_cross_h = np.array( [[1, 1, 1 ], 
                             [0, 0, 0],
                             [-1, -1, -1]])

g_vertical = cv2.filter2D(img_array, -1, prewitt_cross_v)
exp_g_vertical = cv2.equalizeHist(g_vertical)
Image.fromarray(exp_g_vertical).convert('L').save('files/1/prewitt_vertical.jpg')

g_horizontal = cv2.filter2D(img_array, -1, prewitt_cross_h) 
exp_g_horizontal = cv2.equalizeHist(g_horizontal)
Image.fromarray(exp_g_horizontal).convert('L').save('files/1/prewitt_horizontal.jpg')

g_module = np.sqrt(np.square(g_horizontal) + np.square(g_vertical)).astype(np.uint8)
exp_g_module = cv2.equalizeHist(g_module)
Image.fromarray(exp_g_module).convert('L').save('files/1/prewitt_module.jpg')

g_angle = np.arctan(g_vertical/g_horizontal).astype(np.uint8)
exp_g_angle = cv2.equalizeHist(g_angle)
Image.fromarray(exp_g_angle).convert('L').save('files/1/prewitt_angle.jpg')

In [59]:
#prewitt
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
img_prewittx = cv2.filter2D(img_array, -1, kernelx)
img_prewitty = cv2.filter2D(img_array, -1, kernely)

In [76]:
img_prewittx1 = cv2.filter2D(img_array, -1, kernelx)
img_prewittx2 = ndimage.convolve(img_array, kernelx, mode='constant', cval=0.0)

In [77]:
img_prewittx1.shape, img_prewittx2.shape
img_prewittx1

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 3,  3,  3, ...,  0,  0,  0],
       [ 3,  3,  3, ...,  0,  5, 10],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [87, 76, 46, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=uint8)

In [78]:
img_prewittx2

array([[134,  73,  73, ...,  85,  93, 232],
       [254, 253, 253, ...,   0,  15,  16],
       [254, 253, 253, ...,   7, 251, 249],
       ...,
       [  6,  13,  22, ...,   0,   0,   0],
       [196, 180, 210, ...,   0,   0,   0],
       [ 60, 210, 198, ..., 238, 237, 243]], dtype=uint8)

### 3) Применить к изображению фильтр Гаусса с различными значениями среднеквадратического отклонения sigma:

In [46]:
for sigma in (0.5, 5, 10, 20):
    image = gray.filter(ImageFilter.GaussianBlur(sigma))
    image.save(f"files/3/gaussian_filter_sigma_{sigma}.jpg")

### 5) Применить к изображению фильтр лапласиана гауссиан (LoG) с различными размером окна k и значениями среднеквадратического отклонения sigma:
Источник: https://math.stackexchange.com/questions/2445994/discrete-laplacian-of-gaussian-log

Источник: https://homepages.inf.ed.ac.uk/rbf/HIPR2/zeros.htm

#### а) Ядро k = 5, sigma = 0.2

In [47]:
# Проверка для смены типа
np.array_equal(img_array, img_array.astype(np.int16))

True

In [48]:
def LOG(sigma, x, y):
    laplace = -1/(np.pi*sigma**4)*(1-(x**2+y**2)/(2*sigma**2))*np.exp(-(x**2+y**2)/(2*sigma**2))
    return laplace

def LOG_discrete(sigma, n):
    l = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            l[i,j] = LOG(sigma, (i-(n-1)/2),(j-(n-1)/2))
    return l

In [49]:
def LOG_process(k, sigma):
    results = ndimage.convolve(img_array.astype(np.int16), LOG_discrete(sigma, k))

    binary_results = results.copy()
    binary_results[binary_results > 0] = 1
    binary_results[binary_results < 0] = -1

    zero_crossing_results = results.copy()
    zero_crossing_results[zero_crossing_results > 0] = -1
    zero_crossing_results[zero_crossing_results < 0] = -1
    zero_crossing_results[zero_crossing_results == 0] = 0
    return results, binary_results, zero_crossing_results

In [50]:
k, sigma = 5, 0.2
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

In [51]:
k, sigma = 10, 2
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

In [52]:
k, sigma = 30, 2
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

In [53]:
k, sigma = 30, 10
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

### 6. Применить к изображению фильтр разности гауссиан (DoG) с различными параметрами sigma1 и alpha = sigma1 / sigma2
##### Источник: https://gist.github.com/leonidk/8798fdbf38db120b8536d25ea2f8c3b4

In [54]:
def DOG(img, sigma1, alpha):
    sigma2 = sigma1 / alpha
    s1 = gaussian_filter(img, sigma1)
    s2 = gaussian_filter(img, sigma2)

    dog = s1 - s2
    return dog

In [55]:
Image.fromarray(DOG(img_array, 2, 1.6)).save(f"files/6/dog_sigma1_{2}_alpha_{1.6}.jpg")
Image.fromarray(DOG(img_array, 5, 1.6)).save(f"files/6/dog_sigma1_{5}_sigma_{1.6}.jpg")
Image.fromarray(DOG(img_array, 2, 5)).save(f"files/6/dog_sigma1_{2}_sigma_{5}.jpg")
Image.fromarray(DOG(img_array, 2, 10)).save(f"files/6/dog_sigma1_{2}_sigma_{10}.jpg")

### 7. Выделить границы на изображении

#### а) Детектор Марра-Хилдрета

##### https://github.com/adl1995/edge-detectors/blob/master/marr-hildreth-edge.py